In [4]:
!wget -nc 'http://www.capo.gov.cy/capo/capo.nsf/All/BD7C94C6B96F36D8C2257E5A00385E12/$$file/BENEFICIARIES_2014_FINAL_v3.xlsx'

--2016-05-06 11:07:37--  http://www.capo.gov.cy/capo/capo.nsf/All/BD7C94C6B96F36D8C2257E5A00385E12/$file/BENEFICIARIES_2014_FINAL_v3.xlsx
Resolving www.capo.gov.cy... 212.31.118.26
Connecting to www.capo.gov.cy|212.31.118.26|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3869432 (3.7M) [application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]
Saving to: 'BENEFICIARIES_2014_FINAL_v3.xlsx'

BENEFICIARIES_2014_ 100%[===================>]   3.69M   549KB/s    in 7.3s    

2016-05-06 11:07:44 (519 KB/s) - 'BENEFICIARIES_2014_FINAL_v3.xlsx' saved [3869432/3869432]



In [5]:
import pandas as pd

In [8]:
df_raw = pd.read_excel('BENEFICIARIES_2014_FINAL_v3.xlsx')
original_columns = list(df_raw.columns)
translations = ('CODE MEASURE (ABB CODE)',
               'CODE EPEXIGISIS MEASURE',
               'MEASURE', 'CODE APPLICANT', 'NAME / TRADE NAME', 'EAGF', 'EAFRD', 'POSTAL CODE', 'COMMUNITY / DISTRICT')
df_raw = df_raw.rename(columns=dict(zip(original_columns, translations)))
df_raw.head()

,CODE MEASURE (ABB CODE),CODE EPEXIGISIS MEASURE,MEASURE,CODE APPLICANT,NAME / TRADE NAME,EAGF,EAFRD,POSTAL CODE,COMMUNITY / DISTRICT
0,50301020000011,I.2,Εκταρική Επιδότηση 2013 0% - SAPS,NaN,A & F PELOPIDA LTD,2000.00,NaN,8820,ΠΟΛΗ ΧΡΥΣΟΧΟΥΣ
1,50301020000012,I.2,Εκταρική Επιδότηση 2013 0% -SAPS,NaN,A & F PELOPIDA LTD,38984.33,NaN,8820,ΠΟΛΗ ΧΡΥΣΟΧΟΥΣ
2,50301020000011,I.2,Εκταρική Επιδότηση 2013 0% - SAPS,NaN,A & X (ΓΕΩΡΓΙΚΕΣ ΕΠΙΧΕΙΡΗΣΕΙΣ) ΛΙΜΙΤΕΔ,2000.00,NaN,7643,ΚΑΛΟ ΧΩΡΙΟ ΛΑΡΝΑΚΑΣ
3,50301020000012,I.2,Εκταρική Επιδότηση 2013 0% -SAPS,NaN,A & X (ΓΕΩΡΓΙΚΕΣ ΕΠΙΧΕΙΡΗΣΕΙΣ) ΛΙΜΙΤΕΔ,152.04,NaN,7643,ΚΑΛΟ ΧΩΡΙΟ ΛΑΡΝΑΚΑΣ
4,50301020000012,I.2,Εκταρική Επιδότηση 2013 0% -SAPS,NaN,A & X (ΓΕΩΡΓΙΚΕΣ ΕΠΙΧΕΙΡΗΣΕΙΣ) ΛΙΜΙΤΕΔ,37936.49,NaN,7643,ΚΑΛΟ ΧΩΡΙΟ ΛΑΡΝΑΚΑΣ


In [14]:
df_raw.dtypes

CODE MEASURE (ABB CODE)      int64
CODE EPEXIGISIS MEASURE     object
MEASURE                     object
CODE APPLICANT              object
NAME / TRADE NAME           object
EAGF                       float64
EAFRD                      float64
POSTAL CODE                  int64
COMMUNITY / DISTRICT        object
dtype: object

In [13]:
df_raw.pivot_table(index=['CODE EPEXIGISIS MEASURE', 'CODE MEASURE (ABB CODE)'], values=['EAGF', 'EAFRD'], aggfunc=sum)

EAFRD          EAGF
CODE EPEXIGISIS MEASURE CODE MEASURE (ABB CODE)                          
I.2                     50301020000007                  NaN  9.847000e+02
                        50301020000008                  NaN -1.272219e+04
                        50301020000009                  NaN  1.667360e+04
                        50301020000011                  NaN  2.426848e+07
                        50301020000012                  NaN  2.739116e+07
III.3                   50208120000021                  NaN  4.826372e+04
III.4                   50212083120041                  NaN  5.156181e+04
                        50212083120045                  NaN  3.954082e+04
                        50212083120046                  NaN  3.926194e+04
                        50212083120050                  NaN  8.514205e+04
III.5                   50208030000045                  NaN  2.102448e+05
                        50208030000046                  NaN  5.905083e+05
                        50208030000049                  NaN  3.553800e+04
                        50208110000032                  NaN  1.641568e+04
III.6                   50209080000001                  NaN  7.266000e+03
                        50209080000002                  NaN  2.767960e+06
                        50209080000006                  NaN  1.889793e+05
                        50209080000007                  NaN  1.065919e+06
                        50209080000012                  NaN  9.340000e+04
III.7                   50215062320015                  NaN -4.817000e+01
                        50215062320017                  NaN  9.034898e+04
V/B.1.10                50405011251001             95872.70           NaN
V/B.1.13                50405011321001            319092.68           NaN
                        50405011331001             47460.38           NaN
V/B.1.2                 50405011121001           1989264.47           NaN
V/B.1.3                 50405011131001           2110842.01           NaN
V/B.1.6                 50405011211001           6023317.63           NaN
V/B.1.8                 50405011231001           1599597.00           NaN
V/B.2.1                 50405012111001            771276.24           NaN
V/B.2.12                50405012251001             54655.51           NaN
                        50405012261001            458728.74           NaN
V/B.2.2                 50405012121001           4805188.19           NaN
V/B.2.4                 50405012141001           9283770.25           NaN
V/B.2.7                 50405012211001            515176.40           NaN
V/B.2.9                 50405012231001             36542.44           NaN
V/B.3.3                 50405013211001           1249859.23           NaN
V/B.3.5                 50405013221001              4073.27           NaN
V/B.3.6                 50405013131001             23632.55           NaN
                        50405013231001            249368.12           NaN
V/B.4.1                 50405014111001            199647.00           NaN
V/B.4.4                 50405014211001            132351.96           NaN
V/B.4.5                 50405014311001            367404.16           NaN
V/B.5                   50405015111001            349750.50           NaN
VII                     670200000000001                 NaN -2.064861e+05
                        670200000000002                 NaN -2.342967e+05
                        670200000000012                 NaN  2.137290e+03
                        670200000000014                 NaN -5.906539e+04
                        770200000000001                0.00 -5.162153e+04
                        770200000000002                0.00 -1.353574e+05
                        770200000000008                0.00 -3.940000e+00
                        770200000000009                0.00 -5.200000e-01
                        770200000000010                0.00  3.760000e+00
                        770200000000011                0.0

In [12]:
df_raw['CODE APPLICANT'].value_counts().sum()

46288

In [15]:
list(df_raw.columns)

['CODE MEASURE (ABB CODE)',
 'CODE EPEXIGISIS MEASURE',
 'MEASURE',
 'CODE APPLICANT',
 'NAME / TRADE NAME',
 'EAGF',
 'EAFRD',
 'POSTAL CODE',
 'COMMUNITY / DISTRICT']

In [28]:
# There only ever is one value filled EAGF or EAFRD
((df_raw[['EAGF', 'EAFRD']].fillna(0.0) == 0.0).sum(1) > 1).any()

False

In [34]:
from slugify import slugify


df_raw = df_raw.rename(
    columns={
        'NAME / TRADE NAME': 'recipient_name',
        'POSTAL CODE': 'recipient_postcode',
        'COMMUNITY / DISTRICT': 'recipient_location',
    }
)
df_raw['country'] = 'GR'
df_raw['currency'] = 'EUR'

df_raw['recipient_id'] = df_raw.apply(lambda x: 'BG-%s' % x['CODE APPLICANT'] if pd.notnull(x['CODE APPLICANT']) else
                                        'BG-%s-%s' % (x['recipient_postcode'], slugify(x['recipient_name'])), 1)

df_raw['scheme'] = df_raw.apply(lambda x: '%s-%s' % (x['CODE EPEXIGISIS MEASURE'], x['CODE MEASURE (ABB CODE)']), 1)
df_raw['amount'] = df_raw.apply(lambda x: x[['EAGF', 'EAFRD']].sum(), 1)

df_raw.head()

,CODE MEASURE (ABB CODE),CODE EPEXIGISIS MEASURE,MEASURE,CODE APPLICANT,recipient_name,EAGF,EAFRD,recipient_postcode,recipient_location,country,currency,recipient_id,scheme,amount
0,50301020000011,I.2,Εκταρική Επιδότηση 2013 0% - SAPS,NaN,A & F PELOPIDA LTD,2000.00,NaN,8820,ΠΟΛΗ ΧΡΥΣΟΧΟΥΣ,GR,EUR,BG-8820-a-f-pelopida-ltd,I.2-50301020000011,2000.00
1,50301020000012,I.2,Εκταρική Επιδότηση 2013 0% -SAPS,NaN,A & F PELOPIDA LTD,38984.33,NaN,8820,ΠΟΛΗ ΧΡΥΣΟΧΟΥΣ,GR,EUR,BG-8820-a-f-pelopida-ltd,I.2-50301020000012,38984.33
2,50301020000011,I.2,Εκταρική Επιδότηση 2013 0% - SAPS,NaN,A & X (ΓΕΩΡΓΙΚΕΣ ΕΠΙΧΕΙΡΗΣΕΙΣ) ΛΙΜΙΤΕΔ,2000.00,NaN,7643,ΚΑΛΟ ΧΩΡΙΟ ΛΑΡΝΑΚΑΣ,GR,EUR,BG-7643-a-x-georgikes-epikheireseis-limited,I.2-50301020000011,2000.00
3,50301020000012,I.2,Εκταρική Επιδότηση 2013 0% -SAPS,NaN,A & X (ΓΕΩΡΓΙΚΕΣ ΕΠΙΧΕΙΡΗΣΕΙΣ) ΛΙΜΙΤΕΔ,152.04,NaN,7643,ΚΑΛΟ ΧΩΡΙΟ ΛΑΡΝΑΚΑΣ,GR,EUR,BG-7643-a-x-georgikes-epikheireseis-limited,I.2-50301020000012,152.04
4,50301020000012,I.2,Εκταρική Επιδότηση 2013 0% -SAPS,NaN,A & X (ΓΕΩΡΓΙΚΕΣ ΕΠΙΧΕΙΡΗΣΕΙΣ) ΛΙΜΙΤΕΔ,37936.49,NaN,7643,ΚΑΛΟ ΧΩΡΙΟ ΛΑΡΝΑΚΑΣ,GR,EUR,BG-7643-a-x-georgikes-epikheireseis-limited,I.2-50301020000012,37936.49


In [36]:
remove_cols = 'CODE MEASURE (ABB CODE)	CODE EPEXIGISIS MEASURE	MEASURE	CODE APPLICANT\tEAGF	EAFRD'.split('\t')
df_raw = df_raw.drop(remove_cols, 1)
df_raw.head()

,recipient_name,recipient_postcode,recipient_location,country,currency,recipient_id,scheme,amount
0,A & F PELOPIDA LTD,8820,ΠΟΛΗ ΧΡΥΣΟΧΟΥΣ,GR,EUR,BG-8820-a-f-pelopida-ltd,I.2-50301020000011,2000.00
1,A & F PELOPIDA LTD,8820,ΠΟΛΗ ΧΡΥΣΟΧΟΥΣ,GR,EUR,BG-8820-a-f-pelopida-ltd,I.2-50301020000012,38984.33
2,A & X (ΓΕΩΡΓΙΚΕΣ ΕΠΙΧΕΙΡΗΣΕΙΣ) ΛΙΜΙΤΕΔ,7643,ΚΑΛΟ ΧΩΡΙΟ ΛΑΡΝΑΚΑΣ,GR,EUR,BG-7643-a-x-georgikes-epikheireseis-limited,I.2-50301020000011,2000.00
3,A & X (ΓΕΩΡΓΙΚΕΣ ΕΠΙΧΕΙΡΗΣΕΙΣ) ΛΙΜΙΤΕΔ,7643,ΚΑΛΟ ΧΩΡΙΟ ΛΑΡΝΑΚΑΣ,GR,EUR,BG-7643-a-x-georgikes-epikheireseis-limited,I.2-50301020000012,152.04
4,A & X (ΓΕΩΡΓΙΚΕΣ ΕΠΙΧΕΙΡΗΣΕΙΣ) ΛΙΜΙΤΕΔ,7643,ΚΑΛΟ ΧΩΡΙΟ ΛΑΡΝΑΚΑΣ,GR,EUR,BG-7643-a-x-georgikes-epikheireseis-limited,I.2-50301020000012,37936.49


In [37]:
df_raw['year'] = 2014

In [39]:
df_raw.to_csv('gr_2014.csv', index=False, encoding='utf-8')